# Грузим библиотеки

In [ ]:
from flask import Flask
from flask import request
import requests
from flask import make_response
import os
import json
from pandas import DataFrame
import traceback


from methods import access
from methods.emoji import emoji
from methods import telegram_bot_methods
from methods import google_maps_methods
from methods import psql_methods
from methods import reply
from dicts import dict_dates



import app



if __name__ == "__main__":

    
    app.application.run()



In [5]:
import re

task_name = ' ! fdfs fdsfsdf fds'
is_main_task = task_name.replace(' ','')[0] == "!"
if is_main_task

True

In [ ]:
X = {'update_id': 201011794, 'message': {'message_id': 159, 'from': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'language_code': 'ru-RU'}, 'chat': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'type': 'private'}, 'date': 1502611993, 'text': '/help', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 5}]}}
X['message']['text'] = '/start'
X['message']['chat']['last_name']

In [ ]:
#тест отправки графика
import psycopg2
from pandas import DataFrame
import pandas as pd
from datetime import datetime
from datetime import timedelta

import numpy as np

#текущая метка времени
def now_str():
    now = datetime.now()
    now_str = str(now.year)+str(now.month if now.month >= 10 else  '0'+str(now.month))+str(now.day if now.day >= 10 else  '0'+str(now.day)) +' '+str(now.hour if now.hour >= 10 else  '0'+str(now.hour)) + str(now.minute if now.minute >= 10 else  '0'+str(now.minute)) + str(now.second if now.second >= 10 else  '0'+str(now.second))
    return now_str


dbname = 'dbr3jigs1op5oo'
port = '5432'
user = 'muwrkppfuyldmk'
host = 'ec2-54-227-252-202.compute-1.amazonaws.com'
password = '4c4eabfcaf92f7289ccfc1a314d04a3c3806db72b1bf12fd5f0f40c410b14355'

PSQL_heroku_keys = {'dbname' : dbname
                    , 'port' : port
                    , 'user' : user
                    , 'host' : host
                    , 'password' : password
                    }


conn = psycopg2.connect("dbname='%(dbname)s' port='%(port)s' user='%(user)s' host='%(host)s' password='%(password)s'" % PSQL_heroku_keys)


# создаем запрос
cur = conn.cursor()

user_id = 13

cur.execute("select * from public.tasks where  user_id = %(user_id)s  and date_task >= date_trunc('month',now()) - INTERVAL '1MONTH' and date_task < date_trunc('month',now())" % {  'user_id' : user_id} )

df_plan_tasks = DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description]).sort_values(by='date_task' , ascending=True)

cur.close() 


In [ ]:
import re
import time
import pymystem3

interest_info = df_plan_tasks[:]

punctuation_pattern = ' |\.$|\. |, |\/|\(|\)|\'|\"|\!|\?|\+|\:'

text = [re.split(punctuation_pattern, x.lower()) for x in  interest_info['task']]
text_meta = text[:]


In [ ]:

import re
import time
import pymystem3
#стеммизируем слова

interest_info = df_plan_tasks[:]

punctuation_pattern = ' |\.$|\. |, |\/|\(|\)|\'|\"|\!|\?|\+|\:'

text = [re.split(punctuation_pattern, x.lower()) for x in  interest_info['task']]
text_meta = text[:]



start = time.time()

mystem = pymystem3 . Mystem ( )

r = 0
for row in text:
    row_new = []
    for word in row:
        word = word.encode('utf8','ignore').decode()
        if len(word) > 3 and '\\u' != word[:2] and '<' != word[:1] and '\\' != word[:1]:
            
            #test
            if r <= 1:
                new_word = mystem.lemmatize (word)[0]
            else:
                new_word = word
            row_new.append(new_word)
        
    
    text[r] = row_new
    r += 1
    

    
    
stop = time.time()
print (u"Время, затраченное lemmatize- %f на обработку  "%(stop - start))




In [ ]:
unique_words = []
velosity_in_words = []
velosity_in_docs = []
num_row = []

all_docs = 0
all_words = 0

r = 0
for row in text:
    all_docs += 1
    for word in row:
        all_words += 1
        
        if word not in unique_words:
            unique_words.append(word)
            velosity_in_words.append(1)
            num_row.append(r)
        else:
            word_index = unique_words.index(word)
            velosity_in_words[word_index] += 1
            
    r += 1
            
        
        
i = 0
for word_val in velosity_in_words:
    velosity_in_words[i] = word_val / all_words
    velosity_in_docs.append(word_val / all_docs)
    i+=1
    
    
stop_words = ['деньги' ,'подготовить' , 'написать' , 'отправить' ,'начать','перенести','ответить']
stop_parts = ['ить' , 'ать']

all_words = len(velosity_in_docs)
prev_step_words = 0
prev_step = 0

cnt_tasks_interest = 8

for step in np.arange(0.0, 1.0, 0.002):
    step_words = 0
    for i in velosity_in_docs:
        if i > step:
            step_words += 1
    if prev_step_words >= cnt_tasks_interest and step_words < cnt_tasks_interest:
        step = prev_step
        break
    
    if step_words <= cnt_tasks_interest:
        break
    
    prev_step = step
    prev_step_words = step_words
main_tasks = []
meta_rows = []

j = 0
for i in velosity_in_docs:
    if i > step - 0.001:
        word = unique_words[j]
        if word[-3:] not in stop_parts and word not in stop_words:
            
            print(unique_words[j] , i)
            main_tasks.append(unique_words[j])
            meta_rows.append(num_row[j])
            
    j +=  1
        
main_tasks = ' '.join(main_tasks)


In [ ]:
main_tasks

#  персональные новости

In [ ]:
import requests 

api_key = 'f45b069a9ab844369005986b34bb6764'
url = 'https://newsapi.org/v2/'

method = 'top-headlines?'

country = 'ru'
category_list = ['business', 'entertainment', 'general', 'health', 'science', 'sports', 'technology']
category = 'science'
language = 'ru'
from_parameter = '2018-08-02'
to_parameter = '2018-08-02'





get_url = url+method+'country='+country+'&category='+category+'&language='+language+'&from_parameter='+from_parameter+'&to_parameter='+to_parameter 
get_url += '&apiKey='+api_key

r = requests.get(get_url)

json_r_news = json.loads(r.text)

json_r_news['totalResults']

In [ ]:
json_r_news['articles'][0]